# Investigating the RIOT API

Using jupyter notebook to investigate the data returned from calls to the RIOT API

## Ensure the correct conda environment and kernel are being used
[following this guide](https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/)


In [1]:
!conda env list 

# conda environments:
#
base                     /Users/chrisknowles/anaconda3
Riot_dev              *  /Users/chrisknowles/anaconda3/envs/Riot_dev
py311                    /Users/chrisknowles/anaconda3/envs/py311



In [2]:
!type python

python is /Users/chrisknowles/anaconda3/envs/Riot_dev/bin/python


 ## Accessing the API
 Create an account at [developer.riotgames.com](https://developer.riotgames.com/) to generate a development API key
 Keys last 24 hours

Get summoner info from SUMMONER-v4 api using summoner name
url = https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ChrisK1987?api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b

In [3]:
import requests

api_key = 'RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b'
name = 'ChrisK1987'
api_SUMMONER_v4 = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
summoner_info_url = f'{api_SUMMONER_v4}{name}?api_key={api_key}'
print(summoner_info_url)

https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/ChrisK1987?api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b


In [4]:
#get request as .json
requests.get(summoner_info_url).json()

{'id': 'ZLvsYYSu3hAbPJ1nRN7ZlQ3L4Y2-QtPyhYpir3yV-gDEy03b',
 'accountId': 'PlzXQ0iYej21s0pN4ckX4pJ6uBXKvMdeAGLy0UREhy9C2pY',
 'puuid': 'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g',
 'name': 'ChrisK1987',
 'profileIconId': 3379,
 'revisionDate': 1699745156000,
 'summonerLevel': 434}

In [5]:
summoner_info = requests.get(summoner_info_url).json()

In [6]:
list(summoner_info.keys())

['id',
 'accountId',
 'puuid',
 'name',
 'profileIconId',
 'revisionDate',
 'summonerLevel']

In [7]:
#can probe the request for specific info using the dictionary key
summoner_info['puuid']

'z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g'

## Using summoner_info to find match history and teamamtes



using MATCH-v5 api to get match IDs for my summoner_info puid and filtered on ranked games

In [8]:
api_MATCH_v5 = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'

In [9]:
summoner_matches_url = f"{api_MATCH_v5}{summoner_info['puuid']}/ids?type=ranked&start=0&count=20&api_key={api_key}"
print(summoner_matches_url)

#added the url from developer.riotgames.com to test my string maniplulation is correct,  
#api_key will need replacing in future to check if a new request is correct
url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g/ids?type=ranked&start=0&count=20&api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b'
print(url)

https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g/ids?type=ranked&start=0&count=20&api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b
https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/z4qLmeJtju0fcmiPCTmo70aapj23Zh1S42i6NCM_n4bdMtUB5ty1K6OTw4zHfw-UQSzk_G4FbA7F_g/ids?type=ranked&start=0&count=20&api_key=RGAPI-c712ee61-5a8e-46ef-9c1d-cc5d5fbef63b


In [10]:
print(summoner_matches_url == url)
requests.get(summoner_matches_url)

True


<Response [200]>

In [11]:
requests.get(summoner_matches_url).json()

['EUW1_6648437306',
 'EUW1_6648386772',
 'EUW1_6648332079',
 'EUW1_6648244689',
 'EUW1_6646666103',
 'EUW1_6645391387',
 'EUW1_6645299672',
 'EUW1_6645164554',
 'EUW1_6644088095',
 'EUW1_6644029263',
 'EUW1_6643922033',
 'EUW1_6643863487',
 'EUW1_6635285868',
 'EUW1_6635214727',
 'EUW1_6635138424',
 'EUW1_6632385393',
 'EUW1_6632301060',
 'EUW1_6632214776',
 'EUW1_6632170555',
 'EUW1_6630873645']

In [12]:
summoner_matches = requests.get(summoner_matches_url).json()

## TODO
- take each match and get teammates summoner_info
- for each teammate, find their summoner_matches
- for each teammates match, find out if they won or lost
 - count each teammates win/lose ratio to get average win / lose ratio for teammates
 - is there a better way to calculate this stat?
  - are there other interesting stats? (e.g no of teammates who placed zero wards)
  